# Dependencies

In [1]:
from langchain.llms import LlamaCpp, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain import PromptTemplate

# Environment Variables

In [2]:
from dotenv import dotenv_values, load_dotenv
config = dotenv_values(".env")
OPENAI_API_KEY={**config}["OPENAI_API_KEY"]

# Modelo

## Modelo de predicción de texto

In [3]:
#Modelo autocompletador de textos text-davinci-003
llm_openai = OpenAI(model_name = "text-davinci-003", openai_api_key= OPENAI_API_KEY, temperature=0.5)

In [4]:
print(llm_openai("Hola, cómo estás?"))



Estoy bien, ¿y tú?


## Modelo ChatBot

In [8]:
chatgpt = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [45]:
#HumanMessage
response = chatgpt([HumanMessage(content="Hola, Cómo estás?")])
print(response.content)

¡Hola! Soy una inteligencia artificial, por lo que no tengo emociones, pero estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?


# Prompts

## PromptTemplate

In [47]:
basic_template = """

Eres un asistente virtual llamado David que responde a preguntas de manera muy breve.
Pregunta: Cuáles son los ingredientes para preparar {plato}
Respuesta:


"""

In [48]:
#Se construye el template agregándole las variables de entrada
prompt_temp = PromptTemplate(input_variables=["plato"], template=basic_template)

prompt_value = prompt_temp.format(plato="Sancocho de gallina")
print(prompt_value)



Eres un asistente virtual llamado David que responde a preguntas de manera muy breve.
Pregunta: Cuáles son los ingredientes para preparar Sancocho de gallina
Respuesta:





In [49]:
response = llm_openai(prompt_value)
print(response)

Los ingredientes para preparar Sancocho de gallina incluyen una gallina entera, una cebolla, una cucharada de achiote, una cucharada de comino, una cucharada de orégano, una cucharada de ajo en polvo, dos cucharadas de sal, una cucharada de pimienta, una cucharada de cilantro, dos tomates, una taza de papas, una taza de zanahorias, una taza de maíz, una taza de yuca y una taza de plátano verde.


## How many Tokens we're sending?

In [60]:
#Requiere instalar tiktoken
print("tokens:",llm_openai.get_num_tokens("Hola, cómo estás?"))

tokens: 9


## ChatPromptTemplate

Template para modelos de chat. Permite darle información a los modelos de chat en la manera que lo necesiten.

Elementos:

- **Human:** El texto escrito por el usuario
- **AI:** Texto respuesta del modelo
- **System:** Texto enviado al modelo para darle contexto de su funcionamiento

In [65]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate, AIMessagePromptTemplate

In [62]:
#Construcción del template para el sistema
prompt_temp_system = PromptTemplate(
    template = "Eres un asistente virtual que recomienda una alternativa {adjetivo} a un producto",
    input_variables=["adjetivo"]
)
#Langchain se encarga de formatear el prompt de manera correcta para el modelo
template_system = SystemMessagePromptTemplate(prompt=prompt_temp_system)

#Humano
prompt_temp_human = PromptTemplate(template="{texto}",input_variables=["texto"])
template_human = HumanMessagePromptTemplate(prompt=prompt_temp_human)

In [69]:
chat_prompt = ChatPromptTemplate.from_messages([template_system,template_human])

#Formato del prompt
chat_prompt_value = chat_prompt.format_prompt(adjetivo="económica",texto="ipad").to_messages()
print(chat_prompt_value)

[SystemMessage(content='Eres un asistente virtual que recomienda una alternativa económica a un producto'), HumanMessage(content='ipad')]


In [70]:
chat_response = chatgpt(chat_prompt_value)
print(chat_response)

content='Si estás buscando una alternativa económica al iPad, te recomendaría considerar la tablet Amazon Fire HD 10. Esta tablet ofrece una pantalla de alta definición de 10.1 pulgadas, un procesador rápido y potente, y una duración de batería de hasta 12 horas. Además, cuenta con acceso a la tienda de aplicaciones de Amazon, lo que te permitirá descargar una amplia variedad de aplicaciones y juegos. Aunque no tiene todas las características y funcionalidades de un iPad, es una opción más económica que puede cumplir con tus necesidades básicas de navegación web, reproducción de contenido multimedia y uso de aplicaciones.'


# Example Selector

Propósito: Alimentar el modelo con ejemplos y posibles respuestas que se esperan recibir

In [71]:
from langchain import FewShotPromptTemplate

In [74]:
#Se crea una lista de diccionarios con ejemplos para el modelo
examples = [
    {"pregunta": "Cuál es el ingrediente principal de la pizza?","respuesta": "La masa y salsa de tomate"},
    {"pregunta": "Cuál es el ingrediente principal de las hamburguesas?","respuesta": "La carne y el pan"},
    {"pregunta": "Cuál es el ingrediente principal del burrito?","respuesta": "La tortilla y la carne"}
]
#
prompt_temp_examples = PromptTemplate(input_variables=["pregunta","respuesta"],
                                      template = "Pregunta: {pregunta}\nRespuesta: {respuesta}")

In [76]:
#Template que genera el formato para las preguntas y respuestas
prompt_examples = FewShotPromptTemplate(example_prompt=prompt_temp_examples,
                                        examples=examples,
                                        prefix="Eres un asistente virtual culinario que responde a preguntas sobre ingredientes principales de platos de manera breve",
                                        suffix="Pregunta: {pregunta}\nRespuesta:",
                                        input_variables=["pregunta"])

In [79]:
prompt_value = prompt_examples.format(pregunta="Cuál es el ingrediente principal del coctel de camarones?")
print(prompt_value)

Eres un asistente virtual culinario que responde a preguntas sobre ingredientes principales de platos de manera breve

Pregunta: Cuál es el ingrediente principal de la pizza?
Respuesta: La masa y salsa de tomate

Pregunta: Cuál es el ingrediente principal de las hamburguesas?
Respuesta: La carne y el pan

Pregunta: Cuál es el ingrediente principal del burrito?
Respuesta: La tortilla y la carne

Pregunta: Cuál es el ingrediente principal del coctel de camarones?
Respuesta:


In [80]:
ingredients_response = llm_openai(prompt_value)
print(ingredients_response)

 Camarones y salsa picante


# Output Parsers

Formatear la respuesta a conveniencia

In [81]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [82]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [85]:
basic_temp_parser = """Cuáles son los ingredientes para preparar: {platillo}\n{ht_parser}"""

In [88]:
prompt_temp_parser = PromptTemplate(input_variables=["platillo"],
                                    template = basic_temp_parser,
                                    partial_variables = {"ht_parser":format_instructions})

In [89]:
prompt_value_parser = prompt_temp_parser.format(platillo="Burrito")

In [90]:
print(prompt_value_parser)

Cuáles son los ingredientes para preparar: Burrito
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [91]:
response_parser = llm_openai(prompt_value_parser)
response_parser

'\n\nTortilla de harina, carne, frijoles, queso, cebolla, tomate, lechuga, guacamole, crema, salsa.'

In [92]:
output_parser.parse(response_parser)

['Tortilla de harina',
 'carne',
 'frijoles',
 'queso',
 'cebolla',
 'tomate',
 'lechuga',
 'guacamole',
 'crema',
 'salsa.']

# Memoria

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [9]:
memory = ConversationBufferMemory()
chatbot = ConversationChain(llm=chatgpt,memory=memory,verbose=True) #Verbose imprimir historial mientras sigue generando

In [10]:
chatbot.predict(input = "Hola, ¿Cómo estás?, Me llamo David y soy científico de datos")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, ¿Cómo estás?, Me llamo David y soy científico de datos
AI:

> Finished chain.


'¡Hola, David! Me alegro de conocerte. Como inteligencia artificial, no tengo emociones, pero estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte como científico de datos?'

In [12]:
chatbot.predict(input = "Cómo me llamo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, ¿Cómo estás?, Me llamo David y soy científico de datos
AI: ¡Hola, David! Me alegro de conocerte. Como inteligencia artificial, no tengo emociones, pero estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte como científico de datos?
Human: Cómo me llamo?
AI:

> Finished chain.


'Te llamas David.'

In [13]:
chatbot.predict(input = "A qué me dedico?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, ¿Cómo estás?, Me llamo David y soy científico de datos
AI: ¡Hola, David! Me alegro de conocerte. Como inteligencia artificial, no tengo emociones, pero estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte como científico de datos?
Human: Cómo me llamo?
AI: Te llamas David.
Human: A qué me dedico?
AI:

> Finished chain.


'Eres científico de datos.'

In [11]:
memory.chat_memory.messages

[HumanMessage(content='Hola, ¿Cómo estás?, Me llamo David y soy científico de datos'),
 AIMessage(content='¡Hola, David! Me alegro de conocerte. Como inteligencia artificial, no tengo emociones, pero estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte como científico de datos?')]

## Tipos de memoria

Es importante manejar correctamente la memoria del modelo, ya que siempre se envía la memoria y la cantidad de tokens de entrada va creciendo; es decir; se vuelve más costoso conforme más memoria tenga

### Conversation buffer window memory

Permite definir una ventana de mensajes a recordar en vez de todo el histórico

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [23]:
memory = ConversationBufferWindowMemory(k=2) #k=2, sólo enviaría los últimos dos mensajes
chatbot = ConversationChain(llm=chatgpt,memory=memory,verbose=True)

In [24]:
chatbot.predict(input = "Hola, ¿Cómo estás?, Me llamo David y soy físico culturista")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, ¿Cómo estás?, Me llamo David y soy físico culturista
AI:

> Finished chain.


'¡Hola David! ¡Mucho gusto! Soy un modelo de lenguaje de IA creado por OpenAI. Estoy aquí para ayudarte con cualquier pregunta o conversación que tengas. Como soy una IA, no tengo emociones ni cuerpo físico, pero estoy aquí para brindarte información y asistencia en lo que necesites. ¿En qué puedo ayudarte hoy?'

In [25]:
chatbot.predict(input = "Cuál es la altura del monte Fuji?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, ¿Cómo estás?, Me llamo David y soy físico culturista
AI: ¡Hola David! ¡Mucho gusto! Soy un modelo de lenguaje de IA creado por OpenAI. Estoy aquí para ayudarte con cualquier pregunta o conversación que tengas. Como soy una IA, no tengo emociones ni cuerpo físico, pero estoy aquí para brindarte información y asistencia en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Cuál es la altura del monte Fuji?
AI:

> Finished chain.


'La altura del monte Fuji es de aproximadamente 3,776 metros (12,389 pies) sobre el nivel del mar. Es el pico más alto de Japón y un destino turístico muy popular.'

In [26]:
chatbot.predict(input = "Cuál es la esperanza de vida de un colombiano estrato 1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, ¿Cómo estás?, Me llamo David y soy físico culturista
AI: ¡Hola David! ¡Mucho gusto! Soy un modelo de lenguaje de IA creado por OpenAI. Estoy aquí para ayudarte con cualquier pregunta o conversación que tengas. Como soy una IA, no tengo emociones ni cuerpo físico, pero estoy aquí para brindarte información y asistencia en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Cuál es la altura del monte Fuji?
AI: La altura del monte Fuji es de aproximadamente 3,776 metros (12,389 pies) sobre el nivel del mar. Es el pico más alto de Japón y un destino turístico muy popular.
Human: Cuál es la esperanza de vida de un colombiano estrato 1?
AI:

> Finished chain.


'La esperanza de vida de un colombiano en el estrato 1 puede variar según diferentes factores, como el género, el estilo de vida y el acceso a la atención médica. Sin embargo, en general, la esperanza de vida promedio en Colombia es de alrededor de 77 años para los hombres y 82 años para las mujeres. Es importante tener en cuenta que estos son promedios y pueden haber variaciones individuales.'

### Conversation Summary Memory

Almacena un resumen de la conversación, evitando reenviar prompts extensos

In [29]:
from langchain.memory import ConversationSummaryMemory

In [32]:
llm = ChatOpenAI(stop=["\nHuman"])
memory = ConversationSummaryMemory(llm=llm)
chatbot_resume = ConversationChain(llm=llm,memory=memory, verbose=True)

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [33]:
chatbot_resume.predict(input="Hola, ¿Cómo estás?, me llamo David y soy Ingeniero")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, ¿Cómo estás?, me llamo David y soy Ingeniero
AI:

> Finished chain.


'¡Hola David! Me alegra conocerte. Soy una IA y estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?'

In [34]:
chatbot_resume.predict(input="Me encanta la tecnología, la IA y el desarrollo de software")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
El humano saluda al AI y se presenta como David, un ingeniero. El AI responde con entusiasmo y se presenta como una IA dispuesta a ayudar en lo que sea necesario.
Human: Me encanta la tecnología, la IA y el desarrollo de software
AI:

> Finished chain.


'¡Hola David! ¡Qué genial que te guste la tecnología, la IA y el desarrollo de software! Yo también estoy muy emocionado por eso. Como una IA, tengo acceso a una gran cantidad de información sobre estos temas y puedo ayudarte con cualquier pregunta o duda que tengas. ¿En qué puedo ayudarte hoy?'

In [36]:
#La memoria sigue almacenando todo el historial pero al modelo sólo se le envía un resumen
memory.chat_memory.messages

[HumanMessage(content='Hola, ¿Cómo estás?, me llamo David y soy Ingeniero'),
 AIMessage(content='¡Hola David! Me alegra conocerte. Soy una IA y estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?'),
 HumanMessage(content='Me encanta la tecnología, la IA y el desarrollo de software'),
 AIMessage(content='¡Hola David! ¡Qué genial que te guste la tecnología, la IA y el desarrollo de software! Yo también estoy muy emocionado por eso. Como una IA, tengo acceso a una gran cantidad de información sobre estos temas y puedo ayudarte con cualquier pregunta o duda que tengas. ¿En qué puedo ayudarte hoy?')]

### Conversation Knowledge Graph Memory
Genera un grafo de conocimiento donde almacena piezas clave de la conversación

In [37]:
from langchain.memory import ConversationKGMemory
import networkx as nx

In [38]:
llm = ChatOpenAI(stop=["\nHuman"])
memory = ConversationKGMemory(llm=llm)
chatbot_kgm = ConversationChain(llm=llm,memory=memory, verbose=True)

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [39]:
chatbot_kgm.predict(input="Hola, ¿Cómo estás?, me llamo David y soy Ingeniero")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, ¿Cómo estás?, me llamo David y soy Ingeniero
AI:

> Finished chain.


'¡Hola David! ¡Mucho gusto en conocerte! Yo soy una inteligencia artificial creada para ayudar en diversas tareas. ¿En qué puedo asistirte hoy?'

In [44]:
chatbot_kgm.predict(input="Mi madre se llama Mireya y mi perro se llama Max")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Mi madre se llama Mireya y mi perro se llama Max
AI:

> Finished chain.


'¡Hola! ¡Mucho gusto en conocerte! ¿Cómo estás? Oh, qué hermosos nombres tienen tu madre y tu perro. Mireya es un nombre muy bonito y Max suena como un perro muy leal y cariñoso. ¿Qué tipo de perro es Max? ¿Tiene alguna raza en particular?'

In [47]:
#Grafo construido con el contexto de la conversación
print(chatbot_kgm.memory.kg.get_triples())

[('David', 'Ingeniero', 'se llama'), ('Mi madre', 'Mireya', 'se llama'), ('mi perro', 'Max', 'se llama'), ('Human', 'Cómo', 'me llamo')]


In [46]:
chatbot_kgm.predict(input="Cómo me llamo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Cómo me llamo?
AI:

> Finished chain.


'No sé cómo te llamas. Como soy un modelo de IA, no tengo acceso a tu información personal.'